# RUN BELOW IF IN COLAB

In [ ]:
!wget https://github.com/bharris12/URP_2021_Programming_Course/raw/main/lecture_5/data.zip
!unzip data.zip
!rm data.zip
!ls data

# Imports

In [ ]:
#Two Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns


#Mathematical libraries
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import scipy as sp
import pandas as pd

%matplotlib inline

## Learning Goals

1. Reading in data and principles of Tidy Data
2. Processing data to prepare for plotting
3. Principles of plotting
    1. Colors
    2. Showing distributions faithfully
    3. Scaling for the medium of communication
4. Matplotlib/Seaborn Grammar
5. Some plotting exercises


Some of this might be review, but in bioinformatics/data analysis data processing and plotting/communicating are the first and last steps of coding a data-anlysis pipeline. 

<img src="https://qph.fs.quoracdn.net/main-qimg-e7fd43c8c36487389f7bf4f19c52ac2d" />

You will be doing plotting in every step of the data analysis pipeline, so getting comfortable with plotting is crucial

Justin's lecture's about statistics have already covered some of the explore and model part of an analysis, and most of the rest of the lectures will be focusing on a few different examples of those. 

## Background/Lecture

### Quick Introduction to the biology in this lecture

Most of the data in this lecture comes from single cell RNA sequencing (scRNAseq). scRNAseq is a relatively new, but rapidly growing technology for assaying 100s or 1000s and even occasionally millions of individual cells' transcriptomes. 


In short, cells can be separated and barcoded using random DNA sequences before sequencing like normal RNAseq in a variety of different ways. Each method has it's tradeoff. But one constant trait of all the data is sparsity. There are lots of 0s in the data because you are getting a very incomplete sampling of the transcriptome. Yet the method can be extremely useful in studying cell type diversity.

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fnprot.2017.149/MediaObjects/41596_2018_Article_BFnprot2017149_Fig1_HTML.jpg?as=webp" width='500px'/>
(Svensson et al 2018)


The data I am including is all from the mouse motor cortex

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQcdINthsxaWhyKIu3qG07_G-ltSLmptR4ZFoGqs1GEVdz1SDbs" />

The data was generated to specifically look at neuron diversity. There are two main types for neurons, **excitatory**, also known as glutamatergic , and **inhibitory**, also known as GABAergic. The excitatory cell types are named by the layers in the brain they are located in, while the inhibitory ones are named by gene markers.

<img src="http://retina.umh.es/webvision/imageswv/BasicCells.jpg" />

A major focus in studying these cell types is learning how they develop and differentiate from each other. In the case of excitatory cells they develop in the column they will exist in and move from the deeper to shallower layers. For inhibitory cells they develop in another part of the brain, known as the Ganglionic Eminence, and migrate to the cortex. The first major split in defining interneurons is whether they originated in the Medial Ganglionic Eminence (MGE) or the Caudal Ganglionic Eminence (CGE).

<img src="https://www.researchgate.net/publication/321972539/figure/fig1/AS:573998834880512@1513863394719/Migratory-streams-originating-from-the-caudal-ganglionic-eminence-CGE-during-mouse.png" />


#### Some terms I use in the data 

Centroid : The average expression of a cell type. Instead of representing a cell type as all of the cells individually I represent cells as centroids sometimes, where I merely take the average expression of each gene for every cell type



### Reading and processing Columnar/Rectangular Data

#### Flat Files

Flat files can be opened and viewed in any text editor or excel or in the command line. Larger files will give Excel, text editors and your computer issues, so sometimes command line tools are necessary. 

Can be directly viewed in the command line:

Print out entire file:
>$ cat file1.txt 

Print out the top 10 lines:
>$ head file1.txt

Print out the top x lines:
>$ head -n x file1.txt

Print out but allow for scrolling back through output (To escape press q)
>$ less file1.txt




The most common file formats are .txt (text file), .tsv (tab separated value), and .csv (comma separated value)

Columnar data always has a something encoded in the file to separate each column. Depending on the tool you are using to read in the data it can be called different things but the most common names are:

* delimiter (delim)
* separator (sep)
* IFS 

The most common separators are:

* '\t' (tab)
* ',' (comma)
* ' ' (space)
* '\n' (newline)


If you are unsure of what the separator is you can use a head to print out the top of the file 

When reading in these files to python the most common/best functions we use are from numpy and pandas


* np.genfromtxt()
* pd.read_csv() 

> If you google ways to read in text files into python you will find many more ways to read in flat files, but these two functions are the most automated.

I will discuss a few examples of when to use which function after I give a little more info about files.



#### Non-flat/binary
1. .xlxs (Excel File Format) pd.read_excel
2. .h5/.h5f5 (Hierarchical Data Format) pd.read_hdf() or h5py
    1. Often used for big data
    2. Integrated file structure for storing multiple tables of data together (both expression and metadata)


##### Examples
**Your computer doesn't actually care about file extensions, so it isn't uncommon to see a file with a different extension that is really just a flat file** 

All the files below have 100 rows of 10 random numbers

In [ ]:
!head -n 1 ./data/file1.txt

In [ ]:
!head -n 1 ./data/file2.txt

In [ ]:
!head -n 1 ./data/file2.csv

The above output the filename is .csv but the file is tab separated

**The computer reads in your data 1 row at a time, and will interpret a lot about the data based on the first row**

In [ ]:
!head -n 1 ./data/file3.csv

With rectangular data that we are generally working with sometimes there is missing data. As you can see in this modified version of file3.csv there are two commas in a row and only 9 numbers. 

In [ ]:
!head -n 2 ./data/file3_1.csv

In [ ]:
np.genfromtxt('./data/file3_1.csv',delimiter=',')[:2,:]

You can see that numpy has replaced that value with a nan. And still interprets the file as having 10 elements in each row

In [ ]:
np.genfromtxt('./data/file3_2.csv',delimiter=',')[:2,:]

If I remove the second comma numpy will throw an error when I read in the data

In [ ]:
np.genfromtxt('./data/file3_2.csv',delimiter=',', skip_header=1)[:2,:]

But if I skip the first row it the file will be read in, just will be 1 row shorter 

All of the examples I have shown so far has been instances where all of the data is numbers, but we often work with both numerical and categorical data. Numpy does not allow for multiple data types in a single array. 

Numpy defaults to reading in data as a numerical value.

In [ ]:
np.genfromtxt('./data/palmer_species.csv',delimiter=',')

If you tell numpy that you have string data (str) then it will make the entire array strings

In [ ]:
np.genfromtxt('./data/palmer_species.csv',delimiter=',',dtype=str)

Instead of using numpy to read in the data you can use pandas dataframe

In [ ]:
pd.read_csv('./data/palmer_data.csv').head()

Pandas creates, what are called dataframes when you read in data. They have 3 main components. An index (the rownames), columns (column names) and values (data). 

The default nature of read_csv() is to interpret the first row of the data as the header, and to treat every column as a part of the data. It just sets the index to [0,nrows).

In the case of the data I read in above, you can see that the first row of output is bolded, but looks like data, not column names. 

In [ ]:
pd.read_csv('./data/palmer_data.csv', header=None).head()

Rather telling pandas that there is no header will read in the data correctly. Now you if you look at the first column, the one labeled 0, it has the same values as the index column. That is because this column is supposed to be the index.

In [ ]:
pd.read_csv('./data/palmer_data.csv', header=None, index_col=0).head()

#### Tidy data

Now you can see that the first column has been moved to the index location in the output.

At this point you might notice and inconsistency between the way pandas is treating the index (rownames) and columns (column names). There is a specific reason for this. 

Pandas subscribes to a data format/philosophy known as tidy data. 

Jeff Leek in his book The Elements of Data Analytic Style summarizes the characteristics of tidy data as the points

1. Each variable you measure should be in one column.
2. Each different observation of that variable should be in a different row.
3. There should be one table for each "kind" of variable.
4. If you have multiple tables, they should include a column in the table that allows them to be linked.

    (Tidy Data Wikipedia Article)[https://en.wikipedia.org/wiki/Tidy_data]
    

If you want to learn more about the penguin data that we are looking you can read about it [here](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)

<img src='https://allisonhorst.github.io/palmerpenguins/man/figures/lter_penguins.png' width='500px'>

#### Review Questions and Mini Exercises

1. When would you use `np.genfromtxt()` vs `pd.read_csv()`

**Answer Here**

2. What are the core principles of tidy data and why does expression data not work well for it?

**Answer Here**

3. Read in diogo_data.csv properly and store as variable named as diogo1_df
    1. None of the data should have Nan's and if it should be read in as a dataframe it should have the correct rownames and column names

In [ ]:
##TODO

4. Read in centroids_1.csv properly and store in variable names centroids_1_df

In [ ]:
#TODO

### Processing data

In [ ]:
!pip install -q palmerpenguins

In [ ]:
from palmerpenguins import load_penguins
df = load_penguins()

#### Groupings need to be listed as a column

In [ ]:
df.head()

You last column is species, a categorical variable. This means that we can use it to separate the data out by species 

In [ ]:
sns.barplot(data=df, x='species', y='bill_length_mm')

#### pd.melt()

But what if I wanted to compare the distributions of different numerical variables. Say see how bill_length and bill_width compare (not within an observation)? 

To do this you need to make the data "tall" using the function pd.melt()

In [ ]:
bills_tall = pd.melt(df[['bill_length_mm', 'bill_depth_mm']])
bills_tall.head()

In [ ]:
sns.barplot(data=bills_tall,x='variable',y='value')

Now, this is great, but you may notice that we have lost the information about which species each observation came from, when making the data tall, you can add another argument to melt that will bring with each value the species

In [ ]:
bills_tall_species = pd.melt(
    df[['bill_length_mm', 'bill_depth_mm', 'species']], id_vars='species')
bills_tall_species.head()

In [ ]:
sns.barplot(data=bills_tall_species, x='variable', y='value', hue='species')

#### pd.concat()

It is extremely common for data to come in separate files. But we need to join the files together. To use pd.concat()

In [ ]:
centroids_numerical = pd.read_csv(
    './data/centroids_numerical.csv', index_col=0)
centroids_numerical.head()

In [ ]:
centroids_metadata = pd.read_csv('./data/centroids_metadata.csv', index_col=0)
centroids_metadata.head()

In [ ]:
centroids = pd.concat([centroids_numerical, centroids_metadata],axis=1)
centroids.head()
#The axis=1 tells the function to stick the columns next to eachother

In [ ]:
split_1 = centroids_metadata.iloc[:50]
split_2 = centroids_metadata.iloc[50:]
centroids_metadata.head()

In [ ]:
pd.concat([split_2,split_1]).head()

This time without the axis=1 it sticks the two dataframes on top of each other

#### Computations on existing columns to create new ones

In [ ]:
centroids.head()

In [ ]:
#Sometimes you need to add columns based on other columns
centroids['Ogt_zscore'] = stats.zscore(centroids['Ogt'])
centroids['Cacna1a_zscore'] = stats.zscore(centroids['Cacna1a'])

In [ ]:
centroids.head()

#### Review questions and mini exercises

1. What is the difference between using and not using axis=1 with pd.conca()?


**Answer Here**

2. Load in example_expression.csv and example_metadata.csv. Then concat them together into a single dataframe named example_df_concat

In [ ]:
##TODO

## Exercise 

Goal of analysis: Calculate differential expression between MGE and CGE 

This test is similar to the T-test, however it is non-parametric. Instead of using the Gaussian or normal distribution as a null we make the data uniformly distributed and compare the ranks of the samples for each gene. This makes it more robust to outliers in the data. 

### Read in and preprocess the data

1. Read in data
2. Make some descriptive plots about the metadata (sample sizes and stuff)
    1. Whatever you think is necessary to understand the data

In [ ]:
#TODO

In [ ]:
#TODO

In [ ]:
#TODO (Make an informative plot

### Calculating Mann Whitney U test statistic:
$U_1 = R_1 - ((n_1 * (n_1 + 1)) /2)$

$U_2 = R_2 - ((n_2 * (n_2 + 1)) /2)$

$U = min(U_1, U_2)$

$n_1$ = sample size of group 1

$n_2$ = sample size of group 2

R is sum of the rank (order) of the values 

In [ ]:
##TODO

### Calculating AUROC from MannWhitney U:

<img src="https://glassboxmedicine.files.wordpress.com/2019/02/roc-curve-v2.png?w=576" />

The AUROC is a metric used to asses the quality of how well we can predict something. For this case we are measuring how well a gene predicts a specific cell type. 

$\large{AUROC = 1 - \frac{U}{n_1 * n_2}}$

*Hint: AUROCs are values between 0 and 1, but mostly should be between .5 and 1, use a plot to check your results*

In [ ]:
##TODO

In [ ]:
##TODO

### Calculate Log2FC

You calculate log2FC by subtracting the average expressions from each other for each gene. Then taking the log2 of the difference. 

Notes:
* You need to store the sign because log doesn't take negative numbers
* You need to add a psuedocount (1) because log doesn't take 0

In [ ]:
##TODO

### Create Volcano Plots

Volcano plots are a common way of displaying results from differential expression. On the X axis you plot the Log2FC and on the Y axis you can plot the AUROC or log10(P value).

In [ ]:
##TODO

### Calculating P values (Optional):

$\large{Z = \frac{|U - \frac{n_1 * n_2}{2}|}{\sqrt{\frac{n_1 * n_2 * (n_1 + n_2 + 1)}{12}}}}$


In [ ]:
##TODO

Converting the U statistic is making the values normally distributed so you can then convert those to p values using a normal distribution. 

<img src="http://www.z-table.com/uploads/2/1/7/9/21795380/7807141_orig.png" />

In [ ]:
##TODO

### What if we could have predicted this result?

A post-doc in the Gillis lab (Maggie Crow) analyzed hundreds of results of differential expression experiments and showed that certain genes are more often differentially expressed than others. 

<img src="https://www.pnas.org/content/pnas/116/13/6491/F2.medium.gif" />

As we learned before, AUROCs show how good a prediction is, so the list of genes Maggie came up with has an AUROC of .83, which is exceptionally good. 

This doesn't mean that just because the results are predictable that they aren't interesting, just that it isn't surprising that these results would look like results from other studies

In [ ]:
corrected = sm.stats.multipletests(p,method='fdr_bh')

In [ ]:
results['p_raw'] = p
results['is_sig'] = corrected[0]
results['p_adj'] = corrected[1]

In [ ]:
de_prior = pd.read_csv('./data/mouse_de_prior.csv',index_col=0)

In [ ]:
de_prior.head()

In [ ]:
ax = sns.distplot(
    de_prior.loc[results[results.is_sig].index, 'MF.rank'].dropna(),
    kde=False,
    norm_hist=True)
sns.distplot(de_prior['MF.rank'], ax=ax, kde=False, norm_hist=True)
sns.despine()
plt.show()

## Extra Resources

### SQL and databases

Large tabular data can often use special software for accessing and manipulating data. A common framework for this is known as SQL. Tools like [biomart](http://useast.ensembl.org/biomart/martview/3d270413fa0c7c3dca475d573cbf4897) can be accessed through a web interface or using SQL. SQL can be useful if you need to make repeated or many queries of a bioinformatics database (as an aside R has a package biomarRT for querrying biomart). Using SQL will feel a little similar to pandas.

[SQL Wiki](https://en.wikipedia.org/wiki/SQL)

[Biomart Public SQL server info](https://useast.ensembl.org/info/data/mysql.html)

### Some weird "common file formats"

Some bioinformatics software require specific file formats that are subclasses of other file formats

Here are some examples:

[GCT: A Special Flat Files for Gene Expression](http://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats)

[Loom: A Special HDf5 Files for Single Cell Data](http://linnarssonlab.org/loompy/index.html)

Both of these filetypes utilize flat/hdf5 file formats but format them in specific ways to make it easy for the software that they were created for to parse them